In [1]:
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np

In [2]:
df = pd.read_csv('../data/cleaned/final-cleaned.csv')

In [3]:
df

,category_id,category,articles
0,0,opinion,congress much protect american served country predatory profit college
1,1,world,attack civilian site syria grown frequent u n launch rare inquiry document violation investigation limit scope final report make public new york time inves
2,1,world,week relative calm end wednesday people march large number police deploy gas pepper spray
3,2,politics,president monitor middle east crisis golf club threaten iran implored reporter honorable
4,3,arts,linda ronstadt sound voice air cnn new season doctor kick bbc america
...,...,...,...
2336,4,business,former bos premium cable network talk former vice medium executive making documentary
2337,2,politics,lobby president washington hotel supporter remain unwavering business remain brisk
2338,5,sports,lakers first game bryant daughter gianna die helicopter crash sunday lebron james give heartfelt speech game
2339,5,sports,famed run coach already bar sport year dope violation temporarily ban u center safesport action produce lifetime ban


In [4]:
y = np.array(df.category_id.values)
x = df.articles

In [5]:
cv = CountVectorizer(max_features = 5000)

In [6]:
mtr = cv.fit_transform(x.values.astype('U'))

In [7]:
x = pd.DataFrame(mtr.toarray(),columns=cv.get_feature_names())

In [8]:
x

,21st,49ers,50th,aaron,abandon,abbas,abbott,abc,abdicate,abdul,...,zephyr,zhao,zindani,zion,zionism,zoey,zone,zoning,zuberi,zverev
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2336,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2337,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2338,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2339,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state = 0, shuffle = True)
print(len(x_train))
print(len(x_test))
print(len(y_train))
print(len(y_test))

1638
703
1638
703


In [10]:
y

array([0, 1, 1, ..., 5, 5, 2], dtype=int64)

In [11]:
print("X.shape = ",x.shape)
print("y.shape = ",y.shape)

X.shape =  (2341, 5000)
y.shape =  (2341,)


In [12]:
perform_list = []

In [13]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import make_scorer, roc_curve, roc_auc_score
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB,MultinomialNB,BernoulliNB

In [14]:
def run_model(model_name, est_c, est_pnlty):
    mdl=''
    if model_name == 'Logistic Regression':
        mdl = LogisticRegression()
    elif model_name == 'Random Forest':
        mdl = RandomForestClassifier(n_estimators=100 ,criterion='entropy' , random_state=0)
    elif model_name == 'Multinomial Naive Bayes':
        mdl = MultinomialNB(alpha=1.0,fit_prior=True)
    elif model_name == 'Support Vector Classifer':
        mdl = SVC()
    elif model_name == 'Decision Tree Classifier':
        mdl = DecisionTreeClassifier()
    elif model_name == 'K Nearest Neighbour':
        mdl = KNeighborsClassifier(n_neighbors=10 , metric= 'minkowski' , p = 4)
    elif model_name == 'Gaussian Naive Bayes':
        mdl = GaussianNB()
    oneVsRest = OneVsRestClassifier(mdl)
    oneVsRest.fit(x_train, y_train)
    y_pred = oneVsRest.predict(x_test)
    # Performance metrics
    accuracy = round(accuracy_score(y_test, y_pred) * 100, 2)
    # Get precision, recall, f1 scores
    precision, recall, f1score, support = score(y_test, y_pred, average='micro')
    return [round(accuracy,2), round(precision,2), round(recall,2), round(f1score,2)]
    # print(f'Precision : {precision}')
    # print(f'Recall : {recall}')
    # print(f'F1-score : {f1score}')

    # Add performance parameters to list
    # perform_list.append(dict([('Model', model_name),('Test Accuracy', round(accuracy, 2)),('Precision', round(precision, 2)),('Recall', round(recall, 2)),('F1', round(f1score, 2))]))

In [15]:
ml_score = [run_model('Logistic Regression', est_c=None, est_pnlty=None), 
            run_model('Random Forest', est_c=None, est_pnlty=None), 
            run_model('Multinomial Naive Bayes', est_c=None, est_pnlty=None), 
            run_model('Support Vector Classifer', est_c=None, est_pnlty=None),
            run_model('Decision Tree Classifier', est_c=None, est_pnlty=None),
            run_model('K Nearest Neighbour', est_c=None, est_pnlty=None),
            run_model('Gaussian Naive Bayes', est_c=None, est_pnlty=None)
           ]

In [17]:
model_score = pd.DataFrame(data=ml_score, index=['Logistic Regression', 'Random Forest', 'Multinomial Naive Bayes', 'Support Vector Classifer', 'Decision Tree Classifier', 'K Nearest Neighbour', 'Gaussian Naive Bayes'], columns=['Accuracy', 'Precision', 'Recall', 'F1'])

In [18]:
model_score

,Accuracy,Precision,Recall,F1
Logistic Regression,69.56,0.70,0.70,0.70
Random Forest,69.70,0.70,0.70,0.70
Multinomial Naive Bayes,68.56,0.69,0.69,0.69
Support Vector Classifer,70.13,0.70,0.70,0.70
Decision Tree Classifier,51.49,0.51,0.51,0.51
K Nearest Neighbour,26.46,0.26,0.26,0.26
Gaussian Naive Bayes,38.69,0.39,0.39,0.39


## Hyperparamiter Tuning

In [27]:
def rd_tuning():
    data = {'N_estimator' : [], 'Scores' : []}
    for i in range(5, 150):
        rd = RandomForestClassifier(n_estimators=i ,criterion='entropy' , random_state=0)
        rd_prid = rd.fit(x_train, y_train)
        data['N_estimator'].append(i)
        data['Scores'].append(rd_prid.score(x_test, y_test))
    return pd.DataFrame(data)

In [30]:
rd_tuning_scores = rd_tuning()

,N_estimator,Scores
0,5,0.509246
1,6,0.503556
2,7,0.520626
3,8,0.529161
4,9,0.536273
...,...,...
140,145,0.620199
141,146,0.618777
142,147,0.620199
143,148,0.620199


In [42]:
rd_tuning_scores.describe()

,N_estimator,Scores
count,145.000000,145.000000
mean,77.000000,0.599402
std,42.001984,0.021501
min,5.000000,0.503556
25%,41.000000,0.598862
50%,77.000000,0.605974
75%,113.000000,0.608819
max,149.000000,0.621622


In [49]:
rd_tuning_scores.loc[rd_tuning_scores['Scores'] == 0.618777]

,N_estimator,Scores


In [50]:
new_classifier = RandomForestClassifier(n_estimators=145 ,criterion='entropy' , random_state=0).fit(x_train, y_train)
new_classifier
new_y_pred = new_classifier.predict(x_test)

In [78]:
new_text = ['Equities rose Tuesday in Asia on optimism over talks later in the day between Ukraine and Russia aimed at ending their month-long war while there was further support from another drop in oil prices']

In [79]:
y_pred2 = cv.transform(new_text)
yy2 = new_classifier.predict(y_pred2)
result = ""
if yy2 == [0]:
    result = "opinion"
elif yy2 == [1]:
    result = "world"
elif yy2 == [2]:
    result = "Politics News"
elif yy2 == [3]:
    result = "arts"
elif yy2 == [4]:
    result = "business"
elif yy2 == [5]:
    result = "sports"
print(result)

opinion


In [73]:
k = ["Equities rose Tuesday in Asia on optimism over talks later in the day between Ukraine and Russia aimed at ending their month-long war, while there was further support from another drop in oil prices"],
     ["SHANGHAI: Shanghai recorded a steep climb in Covid-19 cases Tuesday as panic-buying in the Chinese city of about 25 million stripped supermarket shelves bare"],
     ["On the morning of March 9, the Smooth Sea 2 oil tanker exploded as it docked at a pier in Samut Prakan province. Owned by petrochemical refiner IRPC, the tanker was carrying 1.8 million litres of petrol and diesel oil from a refinery complex in Rayong"]]

In [132]:
test_set = pd.read_csv('test_set')

In [109]:
test_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 703 entries, 0 to 702
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  703 non-null    int64 
 1   articles    702 non-null    object
dtypes: int64(1), object(1)
memory usage: 11.1+ KB


In [200]:
def predict(test):
    p_result = []
    articles = []
    test_len = len(test.articles)
    for i in range(0, 500):
        y_pred2 = cv.transform([test.articles[i]])
        yy2 = new_classifier.predict(y_pred2)
        if yy2 == [0]:
            result = "opinion"
        elif yy2 == [1]:
            result = "world"
        elif yy2 == [2]:
            result = "Politics News"
        elif yy2 == [3]:
            result = "arts"
        elif yy2 == [4]:
            result = "business"
        elif yy2 == [5]:
            result = "sports"
        p_result.append(result)
        articles =
    return articles

In [201]:
predict(test_set)

[0      televise trial begin smartphone seizure virtua...
 1                start bat cave human activity set loose
 2      exhibition centre national du costume   france...
 3       fund profit handsomely investment stock many ...
 4      secretary state mike pompeo say iraqis celebra...
                              ...                        
 698    best rookie defender n f l dominate speed powe...
 699     north include new delhi plunging temperature ...
 700    plan foil early header late mohamed salah winn...
 701                             new yorkers discuss debt
 702    beltran mets hire november implicated part sig...
 Name: articles, Length: 702, dtype: object,
 0      televise trial begin smartphone seizure virtua...
 1                start bat cave human activity set loose
 2      exhibition centre national du costume   france...
 3       fund profit handsomely investment stock many ...
 4      secretary state mike pompeo say iraqis celebra...
                           

In [191]:
test_set[0:500]

,Unnamed: 0,articles
0,1410,televise trial begin smartphone seizure virtual video witness medium blitz
1,1949,start bat cave human activity set loose
2,9,exhibition centre national du costume france considers dialogue stage runway
3,1117,fund profit handsomely investment stock many description country globe
4,195,secretary state mike pompeo say iraqis celebrate death maj gen qassim suleimani witness say whole story
...,...,...
495,2123,mean john bolton also testify impeachment trial help democrats
496,485,way people generally run president mr bloomberg
497,2228,video show entire meeting lev parnas igor fruman gather president trump donor april
498,965,festival dorrance dance feature performance pop intervention campus


In [168]:
for i in range(1, 500):
    print(test_set.articles[i]

 start bat cave human activity set loose
exhibition centre national du costume   france considers dialogue stage runway
 fund profit handsomely investment stock many description country  globe
secretary state mike pompeo say iraqis celebrate death maj gen qassim suleimani witness say whole story
dylan cozen become fourth yukon breed player draft n h l last year give junior remote hometown path follow
spokeswoman french company answer question edit practice
mayor ponder city hall white house leap
fallout continue inspector general finding bureau botch wiretap application russia investigation
alliance training iraqi soldier battle islamic state say  begin remove troop united state kill iranian general qassim suleimani
san francisco  reach super bowl part counterintuitive strategy instead give ball best player use human decoy
senator lamar alexander say  vote new witness possible death blow democrat case
federer save  match point tennys sandgren fourth set australian open quarterfinal pla